In [1]:
print("Shri Ganesh")

Shri Ganesh


In [20]:
#!pip install cohere
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 55.0 MB/s eta 0:00:00


In [6]:
import cohere
import numpy as np
import pandas as pd
from tqdm import tqdm

In [7]:
text = """
Interstellar is a 2014 epic science fiction drama film directed by Christopher Nolan, who co-wrote the screenplay with his brother Jonathan Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, and Michael Caine.
Set in a dystopian future where Earth is suffering from catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

The screenplay had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg.
Theoretical physicist Kip Thorne was an executive producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar. It was Lynda Obst's final film as producer before her death.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Filming began in late 2013 and took place in Alberta, Klaustur, and Los Angeles.
Interstellar uses extensive practical and miniature effects, and the company DNEG created additional digital effects.

Interstellar was released in theaters on November 7, 2014.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film received generally positive reviews and grossed $681 million worldwide during its initial theatrical run, making it the tenth-highest-grossing film of 2014.
Thorne's computer-generated depiction of a black hole in the film has also received commendation from astronomers and physicists.
Among its various accolades, Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades.
"""

In [10]:
texts = text.split('.')
texts = [t.strip(' \n') for t in texts]

In [11]:
texts

['Interstellar is a 2014 epic science fiction drama film directed by Christopher Nolan, who co-wrote the screenplay with his brother Jonathan Nolan',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, and Michael Caine',
 'Set in a dystopian future where Earth is suffering from catastrophic blight and famine, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
 'The screenplay had its origins in a script Jonathan developed in 2007 and was originally set to be directed by Steven Spielberg',
 'Theoretical physicist Kip Thorne was an executive producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar',
 "It was Lynda Obst's final film as producer before her death",
 'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 'Filming began in late 2013 and took place in Alberta, Klau

Embed the chunk of texts

In [14]:
api_key='<enter your cohere api key>'
co=cohere.Client(api_key)
response = co.embed(texts=texts, input_type="search_document").embeddings

In [16]:
embeds=np.array(response)
print(embeds.shape)

(15, 4096)


Build Search Index

In [22]:
import faiss
dim=embeds.shape[1]
index=faiss.IndexFlatL2(dim)
print(index.is_trained)
index.add(embeds)

True


In [23]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x792e19397c60> >

Search the Index

In [41]:
def search(query, number_of_results=3):
  query_embed=co.embed(texts=[query],input_type='search_query').embeddings[0]
  print("shape of embedded query: ",len(query_embed))
  distances,similar_item_ids=index.search(np.float32([query_embed]),number_of_results)
  texts_np = np.array(texts)
  results= pd.DataFrame(data={'texts':texts_np[similar_item_ids[0]],'distances':distances[0]})
  print(f"Query: '{query}'\nNearest neighbors: ")
  return results

In [43]:
pd.set_option('display.max_colwidth', None)
search("how precise was the science")

shape of embedded query:  4096
Query: 'how precise was the science'
Nearest neighbors: 


,texts,distances
0,"Interstellar uses extensive practical and miniature effects, and the company DNEG created additional digital effects",11976.166016
1,Thorne's computer-generated depiction of a black hole in the film has also received commendation from astronomers and physicists,12197.370117
2,"Theoretical physicist Kip Thorne was an executive producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar",12403.904297


In [44]:
search("when intersteller was launched")

shape of embedded query:  4096
Query: 'when intersteller was launched'
Nearest neighbors: 


,texts,distances
0,"Interstellar was released in theaters on November 7, 2014",8765.234375
1,"Theoretical physicist Kip Thorne was an executive producer and scientific consultant on the film, and wrote the tie-in book The Science of Interstellar",8822.958008
2,"Interstellar is a 2014 epic science fiction drama film directed by Christopher Nolan, who co-wrote the screenplay with his brother Jonathan Nolan",9770.546875
